In [2]:
#import library
from scipy.stats import norm
import numpy as np

import tensorflow as tf
import tensorflow_probability as tfp

tfd = tfp.distributions
import scipy.stats as st
from mpl_toolkits.mplot3d import Axes3D
import math
import matplotlib.pyplot as plt

import seaborn as sns
import pandas as pd
%matplotlib inline

from Inference import MCMC,VI
from Banana import Banana_dist

In [4]:
# Define Matrix and Coev
mu =tf.constant([0,0],dtype = tf.float32)
cov = tf.eye(2, dtype = tf.float32)

In [5]:
#Likelihood
c = 0 # mean of observations
N = 100 # number of observation data
sigma2y = 1 # standard deviation of observation data

## generate the observation data
y_ = np.random.normal(loc =c, scale = sigma2y, size = N)
D = tf.convert_to_tensor(y_,dtype = tf.float32)

def joint_post_prob(D,theta1,theta2):
    z = tf.stack([theta1, theta2], axis=-1)
    
    # define likelihood
    y = tfd.Normal(loc = tf.add(theta2,tf.pow(theta1,2.)), scale = sigma2y)
    
    return(y.prob(D))

In [6]:
#D,Theta1,Theta2

x, y = np.mgrid[-5:5:.01, -5:5:.01]
pos = tf.constant(x.shape + (2,),dtype = tf.float32)
pos[:, :, 0] = x; pos[:, :, 1] = y

post = np.empty(x.shape) ## maybe need tf.convert_to _tensor???

for i in range(100):
    for j in range(100):
        post[i][j] = joint_post_prob(D,pos[i][j][0],pos[i][j][1])

TypeError: 'tensorflow.python.framework.ops.EagerTensor' object does not support item assignment

In [ ]:
# Draw the curve

plt.contour(x, y, post,[joint_post_prob(D,0.,-0.35)])
#plt.xlim(-10,2)
#plt.ylim(-10,1)
plt.xlabel("x1")
plt.ylabel("x2")

In [7]:
# Prior
c = 0 # mean of observations
N = 100 # number of observation data
sigma2y = 1 # standard deviation of observation data

## generate the observation data
y_ = np.random.normal(loc =c, scale = sigma2y, size = N)
D = tf.convert_to_tensor(y_,dtype = tf.float32)

def joint_post_prob(D,theta1,theta2):
    # define random variables prior
    mvn = tfd.MultivariateNormalTriL(
        loc=mu, 
        scale_tril=tf.linalg.cholesky(cov)) 
    z = tf.stack([theta1, theta2], axis=-1)
    
    return(mvn.prob(tf.squeeze(z)))

In [8]:
x, y = np.mgrid[-5:5:.01, -5:5:.01]
pos = np.empty(x.shape + (2,),dtype = np.float32) 
pos[:, :, 0] = x; pos[:, :, 1] = y

post = np.empty(x.shape)

for i in range(100):
    for j in range(100):
        post[i][j] = joint_post_prob(D,pos[i][j][0],pos[i][j][1])

In [ ]:
#plot

plt.contour(x, y, post,[joint_post_prob(D,0.,-0.35)])
#plt.xlim(-10,2)
#plt.ylim(-10,1)
plt.xlabel("x1")
plt.ylabel("x2")